In [1]:
%time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import plot_importance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

CPU times: total: 0 ns
Wall time: 0 ns


In [2]:
%time
df = pd.read_csv('Sample-0004.csv')

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%time
df

CPU times: total: 0 ns
Wall time: 0 ns


,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,1.433158,-0.135424,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
1,1.347939,-0.061015,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
2,1.441331,-0.080833,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
3,1.341744,-0.034491,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
4,1.441067,-0.074840,-0.411514,-0.430135,-0.004761,-0.010526,-0.007826,-0.008587,0.815313,-0.686199,...,-0.710202,-0.575276,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,-1.333181,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34703,0.076511,-0.524380,-0.411514,-0.430135,0.457547,0.087674,0.005299,0.065032,1.169754,1.512796,...,1.369987,-0.120571,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,0.750086,7
34704,-2.564483,2.369977,-0.411514,-0.430135,0.294406,0.350849,1.676032,0.929084,0.992534,1.329547,...,-0.648481,3.051206,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,0.750086,7
34705,-2.564483,2.371282,-0.411514,-0.430135,1.316805,0.291929,0.108340,0.177398,0.992534,1.329547,...,-0.726665,0.336410,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,0.750086,7
34706,-2.564483,2.370748,-0.411514,-0.430135,0.110479,0.152485,0.436717,0.278139,0.992534,1.329547,...,-0.429432,0.336410,-0.104114,-0.104114,-0.345112,-0.178156,-0.087005,-0.057106,0.750086,7


In [4]:
X = df.drop(['Attack'],axis=1).values
y = df.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 0,stratify = y)

In [6]:
features = df.dtypes[df.dtypes != 'object'].index

In [7]:
#Feature engineering
#Feature selection by information gain
from sklearn.feature_selection import mutual_info_classif
importances = mutual_info_classif(X_train, y_train)
# calculate the sum of importance scores
f_list = sorted(zip(map(lambda x: round(x, 4), importances), features), reverse=True)
Sum = 0
fs = []
for i in range(0, len(f_list)):
    Sum = Sum + f_list[i][0]
    fs.append(f_list[i][1])

In [8]:
# select the important features from top to bottom until the accumulated importance reaches 90%
f_list2 = sorted(zip(map(lambda x: round(x, 4), importances/Sum), features), reverse=True)
Sum2 = 0
fs = []
for i in range(0, len(f_list2)):
    Sum2 = Sum2 + f_list2[i][0]
    fs.append(f_list2[i][1])
    if Sum2>=0.9:
        break        


In [9]:
X_fs = df[fs].values
X_fs.shape

(34708, 22)

In [10]:
#Re-split train & test sets after feature selection
X_train, X_test, y_train, y_test = train_test_split(X_fs,y, train_size = 0.8, test_size = 0.2, random_state = 0,stratify = y)

In [11]:
X_train.shape

(27766, 22)

In [12]:
pd.Series(y_train).value_counts()

1    13447
5     6178
7     2740
0     1946
8     1210
9      784
2      651
6      366
3      238
4      206
dtype: int64

In [13]:
from collections import Counter
print("Before Sampling: ",Counter(y_train))

Before Sampling:  Counter({1: 13447, 5: 6178, 7: 2740, 0: 1946, 8: 1210, 9: 784, 2: 651, 6: 366, 3: 238, 4: 206})


In [14]:
from collections import Counter
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_trainf, y_trainf = smote_tomek.fit_resample(X_train, y_train)

In [15]:
print(sorted(Counter(y_trainf).items()))

[(0, 13398), (1, 13443), (2, 13400), (3, 12978), (4, 13276), (5, 12903), (6, 13431), (7, 13447), (8, 13408), (9, 13152)]


In [16]:
pd.Series(y_trainf).value_counts()

7    13447
1    13443
6    13431
8    13408
2    13400
0    13398
4    13276
9    13152
3    12978
5    12903
dtype: int64

In [19]:

# Define k-fold cross-validation object
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [17]:
%time
from catboost import CatBoostClassifier
cat_clf = CatBoostClassifier()


CPU times: total: 0 ns
Wall time: 0 ns


In [20]:
%time
# Define grid of hyperparameters to search over
cat_param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [1, 3, 5]
}

# Perform grid search for best hyperparameters using cross-validation
cat_grid_search = GridSearchCV(estimator=cat_clf, param_grid=cat_param_grid, scoring='accuracy', n_jobs=-1, cv=cv)
cat_grid_search.fit(X_trainf, y_trainf)

# Print best hyperparameters and corresponding score
print("Best hyperparameters for CatBoost:", cat_grid_search.best_params_)
print("Best cross-validation score for CatBoost:", cat_grid_search.best_score_)

CPU times: total: 0 ns
Wall time: 0 ns
0:	learn: 1.8178308	total: 281ms	remaining: 1m 23s
1:	learn: 1.5733123	total: 434ms	remaining: 1m 4s
2:	learn: 1.3971493	total: 587ms	remaining: 58.2s
3:	learn: 1.2700658	total: 731ms	remaining: 54.1s
4:	learn: 1.1666041	total: 877ms	remaining: 51.7s
5:	learn: 1.0827221	total: 1.02s	remaining: 50.1s
6:	learn: 1.0044705	total: 1.17s	remaining: 49s
7:	learn: 0.9437563	total: 1.32s	remaining: 48.1s
8:	learn: 0.8844299	total: 1.46s	remaining: 47.3s
9:	learn: 0.8326247	total: 1.6s	remaining: 46.5s
10:	learn: 0.7921509	total: 1.75s	remaining: 45.9s
11:	learn: 0.7491673	total: 1.89s	remaining: 45.3s
12:	learn: 0.7094501	total: 2.03s	remaining: 44.8s
13:	learn: 0.6733221	total: 2.18s	remaining: 44.5s
14:	learn: 0.6442636	total: 2.33s	remaining: 44.3s
15:	learn: 0.6161508	total: 2.48s	remaining: 44s
16:	learn: 0.5905385	total: 2.63s	remaining: 43.7s
17:	learn: 0.5682280	total: 2.77s	remaining: 43.5s
18:	learn: 0.5461064	total: 2.92s	remaining: 43.2s
19:	le

In [21]:
# Train CatBoost model using best hyperparameters and predict on test set
%time
cat_best = cat_grid_search.best_estimator_
cat_best.fit(X_trainf, y_trainf)
y_pred_cat = cat_best.predict(X_test)

CPU times: total: 0 ns
Wall time: 0 ns
0:	learn: 1.8178308	total: 142ms	remaining: 42.4s
1:	learn: 1.5733123	total: 280ms	remaining: 41.7s
2:	learn: 1.3971493	total: 419ms	remaining: 41.4s
3:	learn: 1.2700658	total: 557ms	remaining: 41.2s
4:	learn: 1.1666041	total: 695ms	remaining: 41s
5:	learn: 1.0827221	total: 832ms	remaining: 40.8s
6:	learn: 1.0044705	total: 972ms	remaining: 40.7s
7:	learn: 0.9437563	total: 1.11s	remaining: 40.7s
8:	learn: 0.8844299	total: 1.25s	remaining: 40.5s
9:	learn: 0.8326247	total: 1.39s	remaining: 40.4s
10:	learn: 0.7921509	total: 1.53s	remaining: 40.2s
11:	learn: 0.7491673	total: 1.67s	remaining: 40.1s
12:	learn: 0.7094501	total: 1.81s	remaining: 40s
13:	learn: 0.6733221	total: 1.95s	remaining: 39.8s
14:	learn: 0.6442636	total: 2.09s	remaining: 39.7s
15:	learn: 0.6161508	total: 2.23s	remaining: 39.6s
16:	learn: 0.5905385	total: 2.39s	remaining: 39.8s
17:	learn: 0.5682280	total: 2.54s	remaining: 39.8s
18:	learn: 0.5461064	total: 2.69s	remaining: 39.7s
19:	le

In [ ]:
# Calculate accuracy and confusion matrix for CatBoost model
acc_cat = accuracy_score(y_test, y_pred_cat)
cm_cat = confusion_matrix(y_test, y_pred_cat)
print("Accuracy for CatBoost:", acc_cat)
print("Confusion matrix for CatBoost:\n", cm_cat)

In [17]:
# Train the CatBoost algorithm
import catboost as cbt
cb = cbt.CatBoostClassifier(depth=7, iterations=300, l2_leaf_reg= 1, learning_rate=0.1)
#cb = cbt.CatBoostClassifier()

cb.fit(X_trainf, y_trainf)
y_pred = cb.predict(X_test)
print(classification_report(y_test,y_pred))
print("Accuracy of CatBoost: "+ str(accuracy_score(y_test, y_pred)))
print("Precision of CatBoost: "+ str(precision_score(y_test, y_pred, average='weighted')))
print("Recall of CatBoost: "+ str(recall_score(y_test, y_pred, average='weighted')))
print("Average F1 of CatBoost: "+ str(f1_score(y_test, y_pred, average='weighted')))
print("F1 of CatBoost for each type of attack: "+ str(f1_score(y_test, y_pred, average=None)))
cb_f1=f1_score(y_test, y_pred, average=None)

# # Plot the confusion matrix
# cm=confusion_matrix(y_test,y_pred)
# print(cm)

0:	learn: 1.8164099	total: 298ms	remaining: 1m 29s
1:	learn: 1.5736325	total: 458ms	remaining: 1m 8s
2:	learn: 1.4071458	total: 625ms	remaining: 1m 1s
3:	learn: 1.2796253	total: 793ms	remaining: 58.7s
4:	learn: 1.1794159	total: 942ms	remaining: 55.6s
5:	learn: 1.0935987	total: 1.09s	remaining: 53.6s
6:	learn: 1.0153343	total: 1.25s	remaining: 52.1s
7:	learn: 0.9477214	total: 1.39s	remaining: 50.9s
8:	learn: 0.8944747	total: 1.55s	remaining: 50s
9:	learn: 0.8485415	total: 1.7s	remaining: 49.2s
10:	learn: 0.7974066	total: 1.86s	remaining: 48.9s
11:	learn: 0.7613019	total: 2s	remaining: 48.1s
12:	learn: 0.7208190	total: 2.16s	remaining: 47.7s
13:	learn: 0.6836675	total: 2.31s	remaining: 47.1s
14:	learn: 0.6521150	total: 2.46s	remaining: 46.7s
15:	learn: 0.6234101	total: 2.61s	remaining: 46.3s
16:	learn: 0.5958176	total: 2.76s	remaining: 45.9s
17:	learn: 0.5738243	total: 2.92s	remaining: 45.7s
18:	learn: 0.5522171	total: 3.07s	remaining: 45.3s
19:	learn: 0.5321479	total: 3.21s	remaining: 4